<h3> Training an AI agent to traverse an environment and collect materials in order using value iteration <h3>
<h5> By Ivan Ovcharov & Veronika Valeva <h5>

### Table of Contents

* Introduction
* Why value iteration?
* Environment description
* Python environment


### Introduction

Over the period of 9 weeks, we have been tasked to train an agent to learn over given constraints in a python environment. The project we chose to tackle is something that closely resembles the infamous <strong> frozen lake </strong> environment. With every environment, there are different ways of approaching how an agent's rules may be defined or what strategy may be used for it to <strong> "learn" </strong>.


After delving a bit deeper into what <strong> reinforcement learning </strong> really is, we made the decision that <strong> <i> Value Iteration </i> </strong> would be best suited for our environment and the given conditions/rules we have defined. But why?

### Why value iteration?

For any given state, we first calculate the state-action values for all the possible <strong>actions</strong> from that given state. We then update the value function of that state with the greatest state-action value. The reason we decided not to utilize <i>policy iteration</i> instead, as we thought unnecessary to have calculations of the expected/mean state-action value. For an environment like ours, where no "predictions" must be made, value iteration was the best option at hand.

With value iteration, we'd be able to terminate when the difference between all the new state values and the old state values is a relatively small value. Furthermore, for a grid-like environment, where all of the possible actions and <strong>"reward"</strong> positions are pre-defined, we'd be better off with iterating over all possible states.

 ![Value iteration algorithm](images/value_iteration.png)

### Environment description

We start with defining what an <strong>environment</strong> really stands for. An environment in AI is what is surrounding the agent. The agent can take input from the environment and deliver output.

As mentioned previously, the environment we have chosen to define is something that very closely resembles the <strong> frozen lake </strong> one by OpenGymAI. The rules are as follows:
* The environment is a grid (initially a 5x5 but scaled down to a 2x2/2x3) where there are "ingredients" that the agent must collect.
* The agent is only able to move up, down, left or right, depending on his position on the grid.
* The agent may not go (for example) left, if left is outside of the grid bounds
* The agent starts in a <i> starting state </i> and finishes in an <i> ending state</i>

Once the agent collects all of the ingredients (must be done in the correct order, otherwise => agent restarts), the agent must "leave" by going to the end state. 

The reward system that is utilized is based on getting items in correct order and finishing the game. There are also slight penalties: when agent steps on an empty cell, he loses a total of 0.2 points and when he steps on a cell, containing an ingredient => +1 points. The reason behind this is that the agent can not only learn how to collect the ingredients in the right order, but the -0.2 points serves as a bound that "pushes" the agent to finish the game in less moves.

 ![Value iteration algorithm](images/env.png)

### Python environment


We first start by defnining what kind of states there will be in our environment. In here, we define the empty state, E. Furthermore, we have one representing our imaginary lettuce and cheese, L and C respectively. Lastly, a state indicating the start and end.

In [6]:
from enum import Enum
from random import randint, choice
from copy import copy

E, L, C, START, END = ' ', 'L', 'C','START','END'

We then define the rule set of our environment. This is done within a python class that holds all of the methods needed.

In [15]:
class RestaurantEnvironment():
    def __init__(self, initial_state=None):
        if initial_state is None:
            self.__initial_state = [E for n in range(6)]
            self.__initial_state[0] = START
            self.__initial_state[2] = C 
            self.__initial_state[4] = L
            self.__initial_state[5] = END
            self.__possible_states = []
            self.playerState = [0, ""]
            self.reward = 0
        else:
            self.__initial_state = copy(initial_state)
            self.__state = self.__initial_state


    def reset(self):
        self.__state = self.__initial_state
        return self.__state

    # Based on playerposition (0 to 24), add a letter to the currentWord
    def calculate_curr_word(self, playerPosition):
        if playerPosition == 2:
            return "C"
        elif playerPosition == 4:
            return "L"

    def step(self, playerPosition):
        if (self.calculate_curr_word(playerPosition)) is not None:
            self.playerState[1] += self.calculate_curr_word(playerPosition)
            self.reward += 1
        else:
            self.reward -= 0.1
        self.playerState[0] = playerPosition
        observation = self.__state  # environment is fully observable
        done = self.get_killed_or_live()
        info = {}  # optional debug info
        return observation, done, info

    def render(self):
        BACKGROUND = [
            ' S │   │ C │',
            '───┼───┼───┼',
            '   │ L │ E │',
            '───┼───┼───┼',
        ]
        rendering = copy(BACKGROUND)
        for n, S_n in enumerate(self.__state):
            if S_n != E:
                row = 2 * (n // 5)
                col = 4 * (n % 5) + 1
                line = rendering[row]
                rendering[row] = line[:col] + S_n + line[col + 1:]

        for line in rendering:
            print(line)

    # =========================================================
    # public functions for agent to calculate optimal policy
    # =========================================================

    def get_possible_actions(self):
        if self.playerState[0] == 0:
            return [self.playerState[0] + 1, self.playerState[0] + 3]
        elif self.playerState[0] == 1:
            return [self.playerState[0] - 1, self.playerState[0] + 1, self.playerState[0] + 3]
        elif self.playerState[0] == 2:
            return [self.playerState[0] -1, self.playerState[0] + 3]
        elif self.playerState[0] == 3:
            return [self.playerState[0] - 3, self.playerState[0] + 1]
        elif self.playerState[0] == 4:
            return [self.playerState[0] - 1, self.playerState[0] - 3, self.playerState[0] + 1]
        elif self.playerState[0] == 5:
            return [self.playerState[0] - 3, self.playerState[0] - 1]

    def get_step_probability(self, new_state, new_inventory):
        new_states = self.get_possible_actions()
        current_inventory = self.playerState[1]
        if new_states.__contains__(new_state):
            if new_state == 2: # State 2 contains C
                if new_inventory == current_inventory + "C":
                    return 1
                else:
                    return 0
            elif new_state == 4: # State 4 contains C
                if new_inventory == current_inventory + "L":
                    return 1
                else:
                    return 0
            elif current_inventory == new_inventory:
                return 1
        else:
            return 0

    def get_killed_or_live(self):
        # Reward R(s) for every possible state
        # Current word must be stored somewhere else
        # B, BU, BUR, BURG
        if self.playerState[1] != "C" or "CL":
            return False
        return True

    def get_transition_prob(self, action, new_state, old_state=None):
        # returns the Transition Probability P(s'| s, a)
        # with s = old_state, a = action and s' = new_state

        # # if the game is over, no transition can take place
        # if self.is_done(old_state):
        #     return 0.0
        #
        # # the position of the action must be empty
        if self.get_killed_or_live():
            self.reset()
            return 0.0

        # check if game is done
        if not self.get_killed_or_live():
            self.reset()
            return 1.0

        # game is not done: calculate all possible states of the opponent
        possible_new_states = []
        possible_new_states = self.get_possible_actions()
        # for action in possible_new_states:
        #     possible_new_state = copy(state_after_X)
        #     possible_new_state[action] = O
        #     possible_new_states.append(possible_new_state)
        if new_state not in possible_new_states:
            return 0.0

        # transition is possible, apply strategy:
        # random opponent, probability is 1 / (# of E before placing the new O)
        prob = 1 / (len(possible_new_states))
        return prob


In [16]:
class Game():
    # example of creation of an environment in the default state
    mdp = RestaurantEnvironment()
    mdp.reset()
    mdp.render()
    print(mdp.step(1))
    print(mdp.step(2))
    print(mdp.step(3))
    print(mdp.playerState[0], mdp.playerState[1])
    print(mdp.get_step_probability(4,"CL"))
    print(mdp.get_step_probability(1, "C"))
    print('possible (internal) game states:')


game = Game()


 START │ C │ C │L
───┼───┼───┼
 END │ L │ E │
───┼───┼───┼
(['START', ' ', 'C', ' ', 'L', 'END'], False, {})
(['START', ' ', 'C', ' ', 'L', 'END'], False, {})
(['START', ' ', 'C', ' ', 'L', 'END'], False, {})
3 C
1
0
possible (internal) game states:
